In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
path = "/content/drive/MyDrive/247/project"
os.chdir(path)

Mounted at /content/drive


In [ ]:
import numpy as np
import tensorflow as tf
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv1D, Conv2D
from keras.layers import Conv1DTranspose, Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Lambda
from keras.layers import Activation
from matplotlib import pyplot
from keras import backend

In [ ]:
def custom_activation(output):
  logexpsum = backend.sum(backend.exp(output), axis=-1, keepdims=True)
  result = logexpsum / (logexpsum + 1.0)
  return result

In [ ]:
def define_discriminator(in_shape=(22, 250), n_classes=4):
  # image input (22, 250)
  in_image = Input(shape=in_shape)
  # downsample
  fe = Conv1D(128, 3, padding='same')(in_image)
  fe = LeakyReLU(alpha=0.2)(fe)
  # downsample
  fe = Conv1D(128, 3, padding='same')(fe)
  fe = LeakyReLU(alpha=0.2)(fe)
  # downsample
  fe = Conv1D(128, 3, padding='same')(fe)
  fe = LeakyReLU(alpha=0.2)(fe)
  # flatten feature maps
  fe = Flatten()(fe)
  # dropout
  fe = Dropout(0.4)(fe)
  # output layer nodes
  fe = Dense(n_classes)(fe)
  # supervised output
  c_out_layer = Activation('softmax')(fe)
  # define and compile supervised discriminator model
  c_model = Model(in_image, c_out_layer)
  c_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.0001, beta_1=0.9), metrics=['accuracy'])
  # unsupervised output
  d_out_layer = Lambda(custom_activation)(fe)
  # define and compile unsupervised discriminator model
  d_model = Model(in_image, d_out_layer)
  d_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001, beta_1=0.9))
  return d_model, c_model
 
# define the standalone generator model
def define_generator(latent_dim):
  # latent_dim is a number smaller than 250
  in_lat = Input(shape=(latent_dim,))
  # print("generator in_lat: ", in_lat.shape)
  n_nodes = 5500
  gen = Dense(n_nodes)(in_lat)
  gen = LeakyReLU(alpha=0.2)(gen)
  # print("before reshape: ", gen.shape)
  gen = Reshape((22, 250))(gen)
  # print("after reshape: ", gen.shape)
  gen = Conv1DTranspose(250, 3, padding='same')(gen)
  gen = LeakyReLU(alpha=0.2)(gen)
  gen = Conv1DTranspose(250, 3, padding='same')(gen)
  gen = LeakyReLU(alpha=0.2)(gen)
  model = Model(in_lat, gen)
  return model

def define_gan(g_model, d_model):
  # make weights in the discriminator not trainable
  d_model.trainable = False
  # connect image output from generator as input to discriminator
  gan_output = d_model(g_model.output)
  # define gan model as taking noise and outputting a classification
  model = Model(g_model.input, gan_output)
  # compile model
  opt = Adam(lr=0.0001, beta_1=0.9)
  model.compile(loss='binary_crossentropy', optimizer=opt)
  return model

In [ ]:
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [ ]:
def data_prep(X,y,sub_sample,average,noise):
    
  total_X = None
  total_y = None
  
  # Trimming the data (sample,22,1000) -> (sample,22,500)
  X = X[:,:,0:500]
  print('Shape of X after trimming:',X.shape)
  
  # Maxpooling the data (sample,22,1000) -> (sample,22,500/sub_sample)
  X_max = np.max(X.reshape(X.shape[0], X.shape[1], -1, sub_sample), axis=3)
  
  
  total_X = X_max
  total_y = y
  print('Shape of X after maxpooling:',total_X.shape)
  
  # Averaging + noise 
  X_average = np.mean(X.reshape(X.shape[0], X.shape[1], -1, average),axis=3)
  X_average = X_average + np.random.normal(0.0, 0.5, X_average.shape)
  
  total_X = np.vstack((total_X, X_average))
  total_y = np.hstack((total_y, y))
  print('Shape of X after averaging+noise and concatenating:',total_X.shape)
  
  # Subsampling
  
  for i in range(sub_sample):
      
    X_subsample = X[:, :, i::sub_sample] + \
                        (np.random.normal(0.0, 0.5, X[:, :,i::sub_sample].shape) if noise else 0.0)
        
    total_X = np.vstack((total_X, X_subsample))
    total_y = np.hstack((total_y, y))
      
  
  print('Shape of X after subsampling and concatenating:',total_X.shape)
  print('Shape of y after subsampling and concatenating:',total_y.shape)
  return total_X,total_y

In [ ]:
## Adjusting the labels so that 

# Cue onset left - 0
# Cue onset right - 1
# Cue onset foot - 2
# Cue onset tongue - 3

y_train_valid -= 769
y_test -= 769


## Random splitting and reshaping the data
# First generating the training and validation indices using random splitting

ind_valid = np.random.choice(2115, 375, replace=False)
ind_train = np.array(list(set(range(2115)).difference(set(ind_valid))))

# Creating the training and validation sets using the generated indices
(X_train, X_valid) = X_train_valid[ind_train], X_train_valid[ind_valid] 
(y_train, y_valid) = y_train_valid[ind_train], y_train_valid[ind_valid]


## Preprocessing the dataset
x_train,y_train = data_prep(X_train,y_train,2,2,True)
x_valid,y_valid = data_prep(X_valid,y_valid,2,2,True)
X_test_prep,y_test_prep = data_prep(X_test,y_test,2,2,True)

Shape of X after trimming: (1740, 22, 500)
Shape of X after maxpooling: (1740, 22, 250)
Shape of X after averaging+noise and concatenating: (3480, 22, 250)
Shape of X after subsampling and concatenating: (6960, 22, 250)
Shape of y after subsampling and concatenating: (6960,)
Shape of X after trimming: (375, 22, 500)
Shape of X after maxpooling: (375, 22, 250)
Shape of X after averaging+noise and concatenating: (750, 22, 250)
Shape of X after subsampling and concatenating: (1500, 22, 250)
Shape of y after subsampling and concatenating: (1500,)
Shape of X after trimming: (443, 22, 500)
Shape of X after maxpooling: (443, 22, 250)
Shape of X after averaging+noise and concatenating: (886, 22, 250)
Shape of X after subsampling and concatenating: (1772, 22, 250)
Shape of y after subsampling and concatenating: (1772,)


In [ ]:
def load_real_samples():
  return [x_train, y_train]

# select a supervised subset of the dataset, ensures classes are balanced
def select_supervised_samples(dataset, n_samples=100, n_classes=4):
  X, y = dataset
  X_list, y_list = list(), list()
  n_per_class = int(n_samples / n_classes)
  for i in range(n_classes):
    # get all images for this class
    X_with_class = X[y == i]
    # print(len(X_with_class))
    # choose random instances
    ix = randint(0, len(X_with_class), n_per_class)
    # add to list
    [X_list.append(X_with_class[j]) for j in ix]
    [y_list.append(i) for j in ix]
  return asarray(X_list), asarray(y_list)
 
# select real samples
def generate_real_samples(dataset, n_samples):
  # split into images and labels
  images, labels = dataset
  # choose random instances
  ix = randint(0, images.shape[0], n_samples)
  # select images and labels
  X, labels = images[ix], labels[ix]
  # generate class labels
  y = ones((n_samples, 1))
  return [X, labels], y

def generate_latent_points(latent_dim, n_samples):
  # generate points in the latent space
  z_input = randn(latent_dim * n_samples)
  # reshape into a batch of inputs for the network
  z_input = z_input.reshape(n_samples, latent_dim)
  return z_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
  # generate points in latent space
  z_input = generate_latent_points(latent_dim, n_samples)
  # predict outputs
  images = generator.predict(z_input)
  # create class labels
  y = zeros((n_samples, 1))
  return images, y

# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, c_model, latent_dim, dataset, n_samples=100):
  # prepare fake examples
  X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
  # scale from [-1,1] to [0,1]
  X = (X + 1) / 2.0
  # plot images
  for i in range(100):
    # define subplot
    pyplot.subplot(10, 10, 1 + i)
    # turn off axis
    pyplot.axis('off')
    # plot raw pixel data
    pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
    # save plot to file
    filename1 = 'generated_plot_%04d.png' % (step+1)
    pyplot.savefig(filename1)
    pyplot.close()
  # evaluate the classifier model
  X, y = dataset
  _, acc = c_model.evaluate(X, y, verbose=0)
  print('Classifier Accuracy: %.3f%%' % (acc * 100))
  # save the generator model
  filename2 = 'g_model_%04d.h5' % (step+1)
  g_model.save(filename2)
  # save the classifier model
  filename3 = 'c_model_%04d.h5' % (step+1)
  c_model.save(filename3)
  print('>Saved: %s, %s, and %s' % (filename1, filename2, filename3))

In [ ]:
def train(g_model, d_model, c_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=256):
  best_acc = 0
  best_idx = 0
  acc = []
  c_models = []
  g_models = []
  # select supervised dataset
  X_sup, y_sup = select_supervised_samples(dataset)
  print(X_sup.shape, y_sup.shape)
  # calculate the number of batches per training epoch
  bat_per_epo = int(dataset[0].shape[0] / n_batch)
  # calculate the number of training iterations
  n_steps = bat_per_epo * n_epochs
  # calculate the size of half a batch of samples
  half_batch = int(n_batch / 2)
  print('n_epochs=%d, n_batch=%d, 1/2=%d, b/e=%d, steps=%d' % (n_epochs, n_batch, half_batch, bat_per_epo, n_steps))
  # manually enumerate epochs
  for i in range(n_steps):
    # update supervised discriminator (c)
    [Xsup_real, ysup_real], _ = generate_real_samples([X_sup, y_sup], half_batch)
    c_loss, c_acc = c_model.train_on_batch(Xsup_real, ysup_real)
    # update unsupervised discriminator (d)
    [X_real, _], y_real = generate_real_samples(dataset, half_batch)
    d_loss1 = d_model.train_on_batch(X_real, y_real)
    X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
    d_loss2 = d_model.train_on_batch(X_fake, y_fake)
    # update generator (g)
    X_gan, y_gan = generate_latent_points(latent_dim, n_batch), ones((n_batch, 1))
    g_loss = gan_model.train_on_batch(X_gan, y_gan)
    print('>%d, c[%.3f,%.0f], d[%.3f,%.3f], g[%.3f]' % (i+1, c_loss, c_acc*100, d_loss1, d_loss2, g_loss))
    _, train_acc = c_model.evaluate(x_train, y_train, verbose=0)
    print('Classifier Accuracy: %.3f%%' % (train_acc * 100))

    acc.append(train_acc)
    c_models.append(c_model)
    g_models.append(gan_model)
    if train_acc > best_acc :
      best_acc = train_acc
      best_idx = i
  print("best_acc: ", best_acc)
  # c_models[best_idx].save('gan_models')
  # c_models[best_idx].save('gan_models')
  return gan_model, d_model, c_model

In [ ]:
# create the discriminator models
d_model, c_model = define_discriminator()
# print(d_model.output.shape)
# create the generator
g_model = define_generator(latent_dim=100)
# print(g_model.output.shape)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
gan_model, d_model, c_model = train(g_model, d_model, c_model, gan_model, dataset, latent_dim=100)

/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


(100, 22, 250) (100,)
n_epochs=10, n_batch=256, 1/2=128, b/e=27, steps=270
4/4 [==============================] - 0s 37ms/step
>1, c[8.431,27], d[0.211,1.601], g[0.225]
Classifier Accuracy: 23.276%
4/4 [==============================] - 0s 39ms/step
>2, c[7.843,29], d[0.106,1.617], g[0.221]
Classifier Accuracy: 23.894%
4/4 [==============================] - 0s 71ms/step
>3, c[7.135,30], d[0.056,1.632], g[0.218]
Classifier Accuracy: 24.353%
4/4 [==============================] - 0s 39ms/step
>4, c[6.891,23], d[0.007,1.642], g[0.215]
Classifier Accuracy: 25.014%
4/4 [==============================] - 0s 39ms/step
>5, c[4.933,41], d[0.003,1.654], g[0.213]
Classifier Accuracy: 25.043%
4/4 [==============================] - 0s 35ms/step
>6, c[4.473,40], d[0.004,1.666], g[0.210]
Classifier Accuracy: 25.330%
4/4 [==============================] - 0s 62ms/step
>7, c[5.380,34], d[0.001,1.675], g[0.208]
Classifier Accuracy: 25.704%
4/4 [==============================] - 0s 40ms/step
>8, c[4.072,

KeyboardInterrupt: ignored

In [ ]:
_, train_acc = c_model.evaluate(x_train, y_train, verbose=0)
print('Classifier Accuracy: %.3f%%' % (train_acc * 100))

Classifier Accuracy: 33.951%


In [ ]:
_, test_acc = c_model.evaluate(X_test_prep, y_test_prep, verbose=0)
print('Classifier Accuracy: %.3f%%' % (test_acc * 100))

Classifier Accuracy: 30.869%
